In [1]:
import os
import sys
import json
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler

from opts import parse_opts
from mean import get_mean, get_std
from spatial_transforms import (
    Compose, Normalize, Scale, CenterCrop, CornerCrop, MultiScaleCornerCrop,
    MultiScaleRandomCrop, RandomHorizontalFlip, ToTensor)
from temporal_transforms import LoopPadding, TemporalRandomCrop
from target_transforms import ClassLabel, VideoID
from target_transforms import Compose as TargetCompose
from dataset import get_training_set, get_validation_set, get_test_set
from utils import Logger
from train import train_epoch
from validation import val_epoch
import test
import collections
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from joblib import dump, load
from sklearn import preprocessing
from scipy import stats
from sklearn.metrics import accuracy_score

from models import *

In [2]:
import easydict
opt = easydict.EasyDict({
    "result_path": 'results2',
    "dataset": 'ucf101-music', # 'ucf101',
    "n_classes": 9, 
    "sample_size": 112,
    "sample_duration": 16,
    "initial_scale": 1.0,
    "n_scales": 5,
    "scale_step": 0.84089641525,
    "train_crop": 'corner',
    "learning_rate": 0.1,
    "momentum": 0.9,
    "dampening": 0.9,
    "weight_decay": 0.001,
    "mean_dataset": 'activitynet',
    "no_mean_norm": False,
    "std_norm": False,
    "nesterov": False,
    "optimizer": 'sgd',
    "lr_patience": 10,
    "batch_size": 16,
    "n_epochs": 2,
    "begin_epoch": 1,
    "n_val_samples": 3,
    "ft_begin_index": 5,
    "scale_in_test": 1.0,
    "crop_position_in_test": 'c',
    "no_softmax_in_test": False,
    "no_cuda": False,
    "n_threads": 4,
    "checkpoint": 2,
    "no_hflip": False,
    "norm_value": 1,
    "model": 'resnet',
    "pretained_model_name": 'resnext-101-kinetics',
    "model_depth": 101,
    "resnet_shortcut": 'B',
    "wide_resnet_k": 2,
    "resnext_cardinality": 32,
    "manual_seed": 1,
    'test_subset': 'test',
})
opt.arch = '{}-{}'.format(opt.model, opt.model_depth)
opt.root_path = '/data/qq/CSCE689/'
opt.video_path = opt.root_path + 'video/UCF-101-jpg/'
opt.annotation_path = opt.root_path + 'video/UCF-music-annotation/ucf_binary_music_annotation.json'

In [3]:
# use two gpu devices on the server, you can customize it depending on how many available gpu devices you have
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [4]:
from models import resnext

# construct model architecture
model = resnext.resnet101(
            num_classes=opt.n_classes,
            shortcut_type=opt.resnet_shortcut,
            cardinality=opt.resnext_cardinality,
            sample_size=opt.sample_size,
            sample_duration=opt.sample_duration)

model = model.cuda()
# wrap the current model again in nn.DataParallel / or we can just remove the .module keys.
model = nn.DataParallel(model, device_ids=None)

# load best weight (we can also refit the model on the combined train-val dataset, 
# but here we simple load the weight and do the final testing)
pretrain = torch.load('./results1/save_50.pth')
model.load_state_dict(pretrain['state_dict'])


/home/qq/CSCE689/3D-ResNets-PyTorch/models/resnext.py:121: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


<All keys matched successfully>

In [5]:
from datasets.ucf101 import UCF101

mean = get_mean(opt.norm_value, dataset='kinetics')
std = get_std(opt.norm_value)
norm_method = Normalize(mean, [1,1,1])


spatial_transform = Compose([
    Scale(opt.sample_size),
    CornerCrop(opt.sample_size, 'c'),
    ToTensor(opt.norm_value), norm_method
])

temporal_transform = LoopPadding(opt.sample_duration)
target_transform = VideoID() # ClassLabel()



# get test data
test_data = UCF101(
    opt.video_path,
    opt.annotation_path,
    'testing',
    0,
    spatial_transform=spatial_transform,
    temporal_transform=temporal_transform,
    target_transform=target_transform,
    sample_duration=16)


# wrap test data
test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=opt.batch_size,
    shuffle=False,
    num_workers=opt.n_threads,
    pin_memory=False)


dataset loading [0/1600]
dataset loading [1000/1600]


In [6]:
examples = ['v_BasketballDunk_g01_c02', 
            'v_Drumming_g05_c06', 
            'v_Rafting_g07_c03', 
            'v_PlayingFlute_g07_c04',
            'v_PlayingPiano_g01_c02']

## Examples

In [7]:
tvns = np.load(opt.root_path + 'feature_ucf_all/class_names_ucf_test.npy')
avi_path = "/data/qq/CSCE689/video/UCF-101"
clip_duration_dict = {}
real_prediction_dict = {}
import os
from moviepy.editor import VideoFileClip
for tvn in examples:
    clip = VideoFileClip(os.path.join(avi_path, tvn[2:-8], tvn + ".avi"))
    clip_duration_dict[tvn] = [clip.duration]
    print(clip_duration_dict)
#     real_prediction_dict[tvn] = test_results['results'][tvn][0]

{'v_BasketballDunk_g01_c02': [2.6]}
{'v_BasketballDunk_g01_c02': [2.6], 'v_Drumming_g05_c06': [8.44]}
{'v_BasketballDunk_g01_c02': [2.6], 'v_Drumming_g05_c06': [8.44], 'v_Rafting_g07_c03': [7.28]}
{'v_BasketballDunk_g01_c02': [2.6], 'v_Drumming_g05_c06': [8.44], 'v_Rafting_g07_c03': [7.28], 'v_PlayingFlute_g07_c04': [16.28]}
{'v_BasketballDunk_g01_c02': [2.6], 'v_Drumming_g05_c06': [8.44], 'v_Rafting_g07_c03': [7.28], 'v_PlayingFlute_g07_c04': [16.28], 'v_PlayingPiano_g01_c02': [7.37]}


In [8]:
clip_duration_dict

{'v_BasketballDunk_g01_c02': [2.6],
 'v_Drumming_g05_c06': [8.44],
 'v_Rafting_g07_c03': [7.28],
 'v_PlayingFlute_g07_c04': [16.28],
 'v_PlayingPiano_g01_c02': [7.37]}

In [9]:
# with open('./hw6_results/clip_duration_dict_example.json', 'w') as fp:
#     json.dump(clip_duration_dict, fp)

In [10]:
# predict
clf = load('./hw6_results/logistic_ucf.joblib') 

x_test_1 = np.load(opt.root_path + 'feature_ucf_all/resnext101_avgpool_test.npy')
x_test_2 = np.load(opt.root_path + 'feature_ucf_all/resnet50_avgpool_test.npy')
x_test = np.concatenate([x_test_1, x_test_2], axis=1)
y_test = np.load(opt.root_path + 'feature_ucf_all/class_names_ucf_test.npy')
y_pred_test_raw = clf.predict(x_test)
y_pred_test_prob_raw = clf.predict_proba(x_test)


# get ground-truth split
# name_to_int = {v:k for k,v in test_data.class_names.items()}
# name_to_int = {'Yes': 1, 'No': 0}


split_idx = []
for idx, y_name in enumerate(y_test):
    if idx == 0 or y_name != y_test[idx-1]:
        split_idx.append(idx)

y_pred_test, y_pred_test_prob, y_pred_test_final = {}, {}, {}
for i, split in enumerate(split_idx):
    if i < len(split_idx) - 1:
        y_pred_test[y_test[split]] = y_pred_test_raw[split:split_idx[i+1]]
        y_pred_test_prob[y_test[split]] = y_pred_test_prob_raw[split:split_idx[i+1]]
        y_pred_test_final[y_test[split]] = np.argmax(np.mean(y_pred_test_prob_raw[split:split_idx[i+1]], axis=0))
        
        

In [11]:
len(y_pred_test)

1587

In [12]:
y_test.shape, x_test.shape

((18873,), (18873, 4096))

### Figure

In [13]:
import matplotlib.pyplot as plt
import numpy as np

music = ['PlayingCello',  'PlayingDhol',   'PlayingGuitar',  'PlayingSitar',  'PlayingViolin', 
         'PlayingDaf',    'PlayingFlute',  'PlayingPiano',   'PlayingTabla']

for tvn in examples:
    interval = clip_duration_dict[tvn][0]/list(y_test).count(tvn)
    x = np.arange(0, clip_duration_dict[tvn][0], interval) + interval
    idx = 0 if tvn in music else 1
    y = np.array([pred[idx] for pred in y_pred_test_prob[tvn]])  # np.arange(len(test_data.class_names) + 1)    
    x = x[:len(y)]
    plt.plot(x, y)
    plt.ylim([-0.1, 1.1])
    plt.xlabel ('time/sec')
    plt.ylabel ('pred score for ground truth label')
    plt.title("Ground Truth Label:  " + tvn[2:-8]  + "\n Model Avg. Predict Score:  " + str(np.mean(y))) # str(real_prediction_dict[tvn]['score'])
    plt.savefig("./hw6_results/fig_one_label/" + tvn, bbox_inches='tight')
    plt.close()
#     break

### Json

In [14]:
import numpy as np


music = ['PlayingCello',  'PlayingDhol',   'PlayingGuitar',  'PlayingSitar',  'PlayingViolin', 
         'PlayingDaf',    'PlayingFlute',  'PlayingPiano',   'PlayingTabla']



timeTrueLabel = {}
for tvn in examples:
    interval = clip_duration_dict[tvn][0]/list(y_test).count(tvn)
    x = np.arange(0, clip_duration_dict[tvn][0], interval) + interval
    idx = 0 if tvn in music else 1
    y_one_label = np.array([pred[idx] for pred in y_pred_test_prob[tvn]]) 
    x = x[:len(y_one_label)]    
    timeTrueLabel[tvn] = {tvn[2:-8]: [[str(time), str(y_one_label[idx])] for idx, time in enumerate(x)]}

In [15]:

with open('./hw6_results/fig_one_label/example.json', 'w') as fp:
    json.dump(timeTrueLabel, fp)



## All others

### Extract clip duration

In [16]:
import multiprocessing
import os
from moviepy.editor import VideoFileClip

tvns = np.load(opt.root_path + 'feature_ucf_all/class_names_ucf_test.npy')
avi_path = "/data/qq/CSCE689/video/UCF-101"
clip_duration_dict = {}

n = len(tvns)

def my_func(i):
    try:
        clip = VideoFileClip(os.path.join(avi_path, tvns[i][2:-8], tvns[i] + ".avi"))
        clip_duration_dict[tvns[i]] = [clip.duration]
        return clip_duration_dict
    except:
        print(tvns[i])
        return {}


# pool = multiprocessing.Pool(10)
# my_clip_duration_dict = pool.map(my_func, list(range(len(tvns))))

from joblib import Parallel, delayed
my_clip_duration_dict = Parallel(n_jobs=10)(delayed(my_func)(i) for i in range(len(tvns)))

In [17]:
len(my_clip_duration_dict)

18873

In [18]:
clip_duration_dict = {}
for d in my_clip_duration_dict:
    clip_duration_dict.update(d)
clip_duration_dict

{'v_BasketballDunk_g01_c02': [2.6],
 'v_WallPushups_g03_c01': [4.57],
 'v_BalanceBeam_g05_c01': [3.8],
 'v_Typing_g02_c01': [10.0],
 'v_Biking_g03_c01': [12.68],
 'v_HandstandWalking_g07_c05': [7.96],
 'v_Drumming_g05_c01': [6.8],
 'v_BabyCrawling_g06_c05': [10.32],
 'v_VolleyballSpiking_g03_c04': [3.17],
 'v_Rowing_g05_c03': [16.56],
 'v_SalsaSpin_g02_c02': [13.32],
 'v_RockClimbingIndoor_g02_c03': [12.0],
 'v_HeadMassage_g07_c05': [9.96],
 'v_CricketShot_g05_c05': [2.24],
 'v_HammerThrow_g07_c05': [3.52],
 'v_CricketShot_g05_c04': [2.2800000000000002],
 'v_ThrowDiscus_g06_c03': [3.88],
 'v_SalsaSpin_g04_c06': [13.32],
 'v_Drumming_g03_c04': [6.96],
 'v_Biking_g03_c02': [12.68],
 'v_CricketShot_g06_c01': [4.88],
 'v_Punch_g03_c02': [11.76],
 'v_SkateBoarding_g02_c04': [3.08],
 'v_Drumming_g06_c01': [12.0],
 'v_CliffDiving_g03_c02': [3.84],
 'v_Biking_g02_c07': [8.01],
 'v_HeadMassage_g02_c07': [9.92],
 'v_SkyDiving_g05_c02': [6.8],
 'v_HandstandWalking_g03_c03': [4.2],
 'v_Drumming_g0

In [19]:
len(clip_duration_dict)

1588

In [14]:
# tvns = np.load(opt.root_path + 'feature_ucf_all/class_names_ucf_test.npy')
# avi_path = "/data/qq/CSCE689/video/UCF-101"
# clip_duration_dict = {}
# import os
# from moviepy.editor import VideoFileClip
# i = 0
# for tvn in tvns[:20]:
#     i += 1
#     if i % 100 == 0:
#         print(i)
#     clip = VideoFileClip(os.path.join(avi_path, tvn[2:-8], tvn + ".avi"))
#     clip_duration_dict[tvn] = [clip.duration]
# #     real_prediction_dict[tvn] = test_results['results'][tvn][0]

In [20]:
with open('./hw6_results/clip_duration_dict.json', 'w') as fp:
    json.dump(clip_duration_dict, fp)
    
    

In [22]:
# predict
clf = load('./hw6_results/logistic_ucf.joblib') 

x_test_1 = np.load(opt.root_path + 'feature_ucf_all/resnext101_avgpool_test.npy')
x_test_2 = np.load(opt.root_path + 'feature_ucf_all/resnet50_avgpool_test.npy')
x_test = np.concatenate([x_test_1, x_test_2], axis=1)
y_test = np.load(opt.root_path + 'feature_ucf_all/class_names_ucf_test.npy')
y_pred_test_raw = clf.predict(x_test)
y_pred_test_prob_raw = clf.predict_proba(x_test)


# get ground-truth split
# name_to_int = {v:k for k,v in test_data.class_names.items()}
# name_to_int = {'Yes': 1, 'No': 0}


split_idx = []
for idx, y_name in enumerate(y_test):
    if idx == 0 or y_name != y_test[idx-1]:
        split_idx.append(idx)

y_pred_test, y_pred_test_prob, y_pred_test_final = {}, {}, {}
for i, split in enumerate(split_idx):
    if i < len(split_idx) - 1:
        y_pred_test[y_test[split]] = y_pred_test_raw[split:split_idx[i+1]]
        y_pred_test_prob[y_test[split]] = y_pred_test_prob_raw[split:split_idx[i+1]]
        y_pred_test_final[y_test[split]] = np.argmax(np.mean(y_pred_test_prob_raw[split:split_idx[i+1]], axis=0))
        
        

In [24]:
len(y_pred_test_final)

1587

### Generate prediction plot for each video -- HW6 ensemble ResNext-101 + ResNet 50 + logistic Regression

In [28]:
import matplotlib.pyplot as plt
import numpy as np

music = ['PlayingCello',  'PlayingDhol',   'PlayingGuitar',  'PlayingSitar',  'PlayingViolin', 
         'PlayingDaf',    'PlayingFlute',  'PlayingPiano',   'PlayingTabla']

for tvn in clip_duration_dict:
    if tvn in y_pred_test_prob:
        interval = clip_duration_dict[tvn][0]/list(y_test).count(tvn)
        x = np.arange(0, clip_duration_dict[tvn][0], interval) + interval
        idx = 0 if tvn in music else 1
        y = 1-np.array([pred[idx] for pred in y_pred_test_prob[tvn]])  # np.arange(len(test_data.class_names) + 1)    
        x = x[:len(y)]
        plt.plot(x, y)
        plt.ylim([-0.1, 1.1])
        plt.xlabel ('time/sec')
        plt.ylabel ('pred score for ground truth label')
        plt.title("Ground Truth Label:  " + tvn[2:-8]  + "\n Model Avg. Predict Score:  " + str(np.mean(y))) # str(real_prediction_dict[tvn]['score'])
        plt.savefig("./hw6_results/fig_one_label/" + tvn, bbox_inches='tight')
        plt.close()
#     break

### Generate json file for each video -- HW6 ensemble ResNext-101 + ResNet 50 + logistic Regression

In [29]:
import numpy as np


timeTrueLabel = {}
for tvn in clip_duration_dict:
    if tvn in y_pred_test_prob:
        interval = clip_duration_dict[tvn][0]/list(y_test).count(tvn)
        x = np.arange(0, clip_duration_dict[tvn][0], interval) + interval
        idx = 0 if tvn in music else 1
        y_one_label = 1-np.array([pred[idx] for pred in y_pred_test_prob[tvn]]) 
        x = x[:len(y_one_label)]    
        timeTrueLabel[tvn] = {tvn[2:-8]: [[str(time), str(y_one_label[idx])] for idx, time in enumerate(x)]}

In [30]:
with open('./hw6_results/fig_one_label/timeLabel.json', 'w') as fp:
    json.dump(timeTrueLabel, fp)